In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
%cd gdrive/My Drive

/content/gdrive/My Drive


In [4]:
import os 

if not os.path.isdir('neuro-sat'):
  !git clone --recurse-submodules https://github.com/dmeoli/neuro-sat
else:
  %cd neuro-sat
  !git pull

/content/gdrive/My Drive/neuro-sat


In [5]:
!pip install -r requirements.txt

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 124 kB 9.6 MB/s 
     |████████████████████████████████| 222 kB 14.1 MB/s 
     |████████████████████████████████| 1.6 MB 9.7 MB/s 
     |████████████████████████████████| 3.0 MB 74.6 MB/s 
     |████████████████████████████████| 376 kB 16.2 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.2-py3-none-any.whl size=388142 sha256=ac8befd977bc4b575dd36f96932e68616fbb7f91d692d280637320f78f1ca02a
  Stored in directory: /root/.cache/pip/wheels/55/93/b6/2eeb0465afe89aee74d7a07a606e9770466d7565abd45a99d5
Successfully built torch-geometric


### Split dataset

In [6]:
datasets = {"uniform-random-3-sat": ["uf50-218",
                                     "uf100-430",
                                     "uf250-1065"],
            "graph-coloring": ["flat50-115",
                               "flat100-239",
                               "flat150-360",
                               "flat200-479"]}

In [17]:
for PROBLEM_TYPE in datasets.keys():
  !bash train_val_test_split.sh "$PROBLEM_TYPE"

Copying files: 2100 files [00:14, 144.10 files/s]
Copying files: 1300 files [05:44,  3.77 files/s]


# NeuroSAT

In [ ]:
%cd NeuroSAT

/content/gdrive/My Drive/neuroSAT/NeuroSAT-tf1


In [ ]:
!bash setup.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
The following additional packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pkg-resources python3-secretstorage python3-setuptools python3-six
  python3-wheel python3-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python3-cryptography-vectors
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0
  python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pip python3-pkg-resources python3-secretstorage python3-setuptools
  python3-six python3-wheel p

In [ ]:
%cd ..

/content/gdrive/My Drive/neuroSAT


# AlphaZeroSAT

In [18]:
%cd AlphaZeroSAT

/content/gdrive/My Drive/neuro-sat/AlphaZeroSAT


### Build C++

In [19]:
%cd MCTSminisat
!apt install libgsl-dev
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/My Drive/neuro-sat/AlphaZeroSAT/MCTSminisat
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgsl23 libgslcblas0
Suggested packages:
  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html
The following NEW packages will be installed:
  libgsl-dev libgsl23 libgslcblas0
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 1,926 kB of archives.
After this operation, 9,474 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgslcblas0 amd64 2.4+dfsg-6 [79.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl23 amd64 2.4+dfsg-6 [823 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl-dev amd64 2.4+dfsg-6 [1,023 kB]
Fetched 1,926 kB in 1s (1,952 kB/s)
Selecting previously unselected package libgslcblas0:amd64.
(Reading database ... 160837 files and direct

In [21]:
!python train.py --help

2021-07-31 15:56:18.551036: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
usage: train.py [-h] [--save_dir SAVE_DIR] [--best_model BEST_MODEL]
                [--status_file STATUS_FILE] [--result_file RESULT_FILE]
                [--dump_dir DUMP_DIR] [--dump_file DUMP_FILE]
                [--train_path TRAIN_PATH] [--test_path TEST_PATH]
                [--max_clause MAX_CLAUSE] [--max_var MAX_VAR]
                [--sl_buffer_size SL_BUFFER_SIZE] [--n_batch N_BATCH]
                [--n_stack N_STACK] [--n_repeat N_REPEAT] [--n_start N_START]
                [--n_train_files N_TRAIN_FILES] [--n_test_files N_TEST_FILES]
                [--l2_coeff L2_COEFF] [--sl_num_steps SL_NUM_STEPS]
                [--sl_n_batch SL_N_BATCH] [--sl_n_checkpoint SL_N_CHECKPOINT]
                [--n_cycles N_CYCLES] [--show_only SHOW_ONLY]
                [--which_cycle WHICH_CYCLE]

optional arguments:
  -h, --help            

In [25]:
for (PROBLEM_TYPE, PROBLEMS) in datasets.items():
  for PROBLEM_NAME in PROBLEMS:
    !python train.py \
      --save_dir runs/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
      --best_model -1 \
      --status_file status.pkl \
      --result_file result.pkl \
      --dump_dir parameters \
      --dump_file sl.pkl \
      --train_path ../data/"$PROBLEM_TYPE"/train/"$PROBLEM_NAME" \
      --test_path ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME" \
      --max_clause 120 \
      --max_var 20 \
      --sl_buffer_size 1000000 \
      --n_batch 32 \
      --n_stack 1 \
      --n_repeat 100 \
      --n_start 0 \
      --n_train_files 0 \
      --n_test_files 0 \
      --l2_coeff 0.0001 \
      --sl_num_steps 64000 \
      --sl_n_batch 32 \
      --sl_n_checkpoint 32000 \
      --n_cycles 2 \
      --show_only 'No' \
      --which_cycle 0

2021-07-31 15:59:02.815219: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
best_model is -1
n_start is 0
ev_hist has length 0
length_hist is 0
status_file is runs/uniform-random-3-sat/uf50-218/status.pkl
args is runs/uniform-random-3-sat/uf50-218__../data/uniform-random-3-sat/train/uf50-218__../data/uniform-random-3-sat/test/uf50-218
2021-07-31 15:59:04.665661: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-31 15:59:04.679023: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-31 15:59:04.679608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 device

In [26]:
%cd ..

/content/gdrive/My Drive/neuro-sat


# GraphQSAT

In [7]:
%cd GraphQSAT

/content/gdrive/MyDrive/neuro-sat/GraphQSAT


### Build C++

In [8]:
%cd minisat
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/MyDrive/neuro-sat/GraphQSAT/minisat
rm -f   build/release/minisat/core/Solver.o  build/release/minisat/core/Main.o  build/release/minisat/simp/SimpSolver.o  build/release/minisat/simp/Main.o  build/release/minisat/utils/System.o  build/release/minisat/utils/Options.o  build/release/minisat/gym/GymSolver.o   build/debug/minisat/core/Solver.o  build/debug/minisat/core/Main.o  build/debug/minisat/simp/SimpSolver.o  build/debug/minisat/simp/Main.o  build/debug/minisat/utils/System.o  build/debug/minisat/utils/Options.o  build/debug/minisat/gym/GymSolver.o   build/profile/minisat/core/Solver.o  build/profile/minisat/core/Main.o  build/profile/minisat/simp/SimpSolver.o  build/profile/minisat/simp/Main.o  build/profile/minisat/utils/System.o  build/profile/minisat/utils/Options.o  build/profile/minisat/gym/GymSolver.o   build/dynamic/minisat/core/Solver.o  build/dynamic/minisat/core/Main.o  build/dynamic/minisat/simp/SimpSolver.o  build/dynamic/minisat/simp/Main.o  build/dynam

### Add metadata for evaluation (train and validation set)

In [9]:
for (PROBLEM_TYPE, PROBLEMS) in datasets.items():
  for PROBLEM_NAME in PROBLEMS:
    !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/train/"$PROBLEM_NAME"
    !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$PROBLEM_NAME"

Streaming output truncated to the last 5000 lines.
Rollout 349, steps 91, num_restarts 0.
Rollout 350, steps 33, num_restarts 0.
Rollout 351, steps 17, num_restarts 0.
Rollout 352, steps 27, num_restarts 0.
Rollout 353, steps 16, num_restarts 0.
Rollout 354, steps 22, num_restarts 0.
Rollout 355, steps 44, num_restarts 0.
Rollout 356, steps 26, num_restarts 0.
Rollout 357, steps 9, num_restarts 0.
Rollout 358, steps 29, num_restarts 0.
Rollout 359, steps 46, num_restarts 0.
Rollout 360, steps 16, num_restarts 0.
Rollout 361, steps 52, num_restarts 0.
Rollout 362, steps 70, num_restarts 0.
Rollout 363, steps 30, num_restarts 0.
Rollout 364, steps 35, num_restarts 0.
Rollout 365, steps 17, num_restarts 0.
Rollout 366, steps 14, num_restarts 0.
Rollout 367, steps 65, num_restarts 0.
Rollout 368, steps 27, num_restarts 0.
Rollout 369, steps 25, num_restarts 0.
Rollout 370, steps 56, num_restarts 0.
Rollout 371, steps 32, num_restarts 0.
Rollout 372, steps 38, num_restarts 0.
Rollout 373, s

### Train

In [30]:
!python dqn.py --help

usage: dqn.py [-h] [--lr LR] [--bsize BSIZE] [--eps-init EPS_INIT]
              [--expert-exploration-prob EXPERT_EXPLORATION_PROB]
              [--gamma GAMMA] [--eps-decay-steps EPS_DECAY_STEPS]
              [--target-update-freq TARGET_UPDATE_FREQ]
              [--batch-updates BATCH_UPDATES] [--buffer-size BUFFER_SIZE]
              [--history-len HISTORY_LEN]
              [--init-exploration-steps INIT_EXPLORATION_STEPS]
              [--step-freq STEP_FREQ] [--loss LOSS] [--save-freq SAVE_FREQ]
              [--train-problems-paths TRAIN_PROBLEMS_PATHS]
              [--eval-freq EVAL_FREQ] [--eval-time-limit EVAL_TIME_LIMIT]
              [--status-dict-path STATUS_DICT_PATH] [--core-steps CORE_STEPS]
              [--priority_alpha PRIORITY_ALPHA]
              [--priority_beta PRIORITY_BETA] [--opt OPT]
              [--e2v-aggregator E2V_AGGREGATOR] [--n_hidden N_HIDDEN]
              [--hidden_size HIDDEN_SIZE]
              [--decoder_v_out_size DECODER_V_OUT_SIZE]
   

In [ ]:
for (PROBLEM_TYPE, PROBLEMS) in datasets.items():
  for PROBLEM_NAME in PROBLEMS:
    !python dqn.py \
      --logdir log \
      --env-name sat-v0 \
      --train-problems-paths ../data/"$PROBLEM_TYPE"/train/"$PROBLEM_NAME" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$PROBLEM_NAME" \
      --lr 0.00002 \
      --bsize 64 \
      --buffer-size 20000 \
      --eps-init 1.0 \
      --eps-final 0.01 \
      --gamma 0.99 \
      --eps-decay-steps 30000 \
      --batch-updates 50000 \
      --history-len 1 \
      --init-exploration-steps 5000 \
      --step-freq 4 \
      --target-update-freq 10 \
      --loss mse \
      --opt adam \
      --save-freq 500 \
      --grad_clip 1 \
      --grad_clip_norm_type 2 \
      --eval-freq 1000 \
      --eval-time-limit 3600 \
      --core-steps 4 \
      --expert-exploration-prob 0.0 \
      --priority_alpha 0.5 \
      --priority_beta 0.5 \
      --e2v-aggregator sum \
      --n_hidden 1 \
      --hidden_size 64 \
      --decoder_v_out_size 32 \
      --decoder_e_out_size 1 \
      --decoder_g_out_size 1 \
      --encoder_v_out_size 32 \
      --encoder_e_out_size 32 \
      --encoder_g_out_size 32 \
      --core_v_out_size 64 \
      --core_e_out_size 64 \
      --core_g_out_size 32 \
      --activation relu \
      --penalty_size 0.1 \
      --train_time_max_decisions_allowed 500 \
      --test_time_max_decisions_allowed 500 \
      --no_max_cap_fill_buffer \
      --lr_scheduler_gamma 1 \
      --lr_scheduler_frequency 3000 \
      --independent_block_layers 0

EncoderCoreDecoder(
  (encoder): GraphNet(
    (edge_mlp): Sequential(
      (0): Linear(in_features=2, out_features=32, bias=True)
      (1): ReLU()
      (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (node_mlp): Sequential(
      (0): Linear(in_features=2, out_features=32, bias=True)
      (1): ReLU()
      (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (global_mlp): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU()
      (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (op): ModifiedMetaLayer(
        edge_model=<function GraphNet.__init__.<locals>.edge_model at 0x7f3e8a1e1200>,
        node_model=<function GraphNet.__init__.<locals>.node_model at 0x7f3e8a1e1290>,
        global_model=<function GraphNet.__init__.<locals>.global_model at 0x7f3e8a1e1320>
    )
  )
  (core): GraphNet(
    (edge_mlp): Sequential(
      (0): Linear(in_features=384, out_features=64, bias=True)
      (1):

### Add metadata for evaluation (test set)

In [ ]:
for (PROBLEM_TYPE, PROBLEMS) in datasets.items():
  for PROBLEM_NAME in PROBLEMS:
    !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME"

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/neuro-sat/data/uniform-random-3-sat/test/uf50-218/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 38, num_restarts 0.
Rollout 2, steps 16, num_restarts 0.
Rollout 3, steps 8, num_restarts 0.
Rollout 4, steps 15, num_restarts 0.
Rollout 5, steps 87, num_restarts 0.
Rollout 6, steps 17, num_restarts 0.
Rollout 7, steps 13, num_restarts 0.
Rollout 8, steps 27, num_restarts 0.
Rollout 9, steps 29, num_restarts 0.
Rollout 10, steps 28, num_restarts 0.
Rollout 11, steps 17, num_restarts 0.
Rollout 12, steps 40, num_restarts 0.
Rollout 13, steps 67, num_restarts 0.
Rollout 14, steps 17, num_restarts 0.
Rollout 15, steps 66, num_restarts 0.
Rollout 16, steps 62, num_restarts 0.
Rollout 17, steps 38, num_restarts 0.
Rollout 18, steps 16, num_restarts 0.
Rollout 19, steps 32, num_restarts 0.
Rollout 20, steps 94, num_restarts 0.
Rollo

### Evaluate

In [ ]:
!python evaluate.py --help

usage: evaluate.py [-h] [--core-steps CORE_STEPS]
                   [--eval-time-limit EVAL_TIME_LIMIT] [--with_restarts]
                   [--no_restarts] [--compare_with_restarts]
                   [--compare_no_restarts]
                   [--test_max_data_limit_per_set TEST_MAX_DATA_LIMIT_PER_SET]
                   [--test_time_max_decisions_allowed TEST_TIME_MAX_DECISIONS_ALLOWED]
                   [--env-name ENV_NAME] [--debug] [--model-dir MODEL_DIR]
                   [--model-checkpoint MODEL_CHECKPOINT] [--logdir LOGDIR]
                   [--eps-final EPS_FINAL]
                   [--eval-problems-paths EVAL_PROBLEMS_PATHS]
                   [--train_max_data_limit_per_set TRAIN_MAX_DATA_LIMIT_PER_SET]
                   [--no-cuda] [--dump_timings_path DUMP_TIMINGS_PATH]

optional arguments:
  -h, --help            show this help message and exit
  --core-steps CORE_STEPS
                        Number of message passing iterations. \-1 for the same
                 

In [ ]:
MODEL_DIR="Jul28_23-07-46_9b921cd36aa9"
CHECKPOINT="model_50000"

In [ ]:
for (PROBLEM_TYPE, PROBLEMS) in datasets.items():
  for PROBLEM_NAME in PROBLEMS:
    for MODEL_DECISION in [10, 50, 100, 300, 500]:
      !python evaluate.py \
        --logdir log \
        --env-name sat-v0 \
        --core-steps -1 \
        --eps-final 0.0 \
        --eval-time-limit 100000000000000 \
        --no_restarts \
        --test_time_max_decisions_allowed "$MODEL_DECISION" \
        --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME" \
        --model-dir runs/"$MODEL_DIR" \
        --model-checkpoint "$CHECKPOINT".chkp \
        >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gqsat.tsv

In [ ]:
%cd ..

/content/gdrive/My Drive/neuro-sat
